In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import xarray as xr
import numpy as np                
import os, subprocess
from shutil import copy2

In [2]:
new = xr.open_mfdataset('/Users/robert/Codes/rte-rrtmgp/examples/rfmip-clear-sky/r??_template.nc')
ref = xr.open_mfdataset('/Users/robert/Dropbox/Scientific/Projects/RFMIP/RFMIP-IRF/LBLRTM/r??.nc')
new.load()
ref.load()
conds = xr.open_dataset('/Users/robert/Codes/rte-rrtmgp/examples/rfmip-clear-sky/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-0-4_none.nc')
conds.load()             
toa = conds.isel(site=0,expt=0).pres_level.argmin()
sfc = conds.isel(site=0,expt=0).pres_level.argmax()
(toa, sfc)

(<xarray.DataArray 'pres_level' ()>
 array(0)
 Coordinates:
     lon      float32 27.0
     lat      float32 -28.5
     time     datetime64[ns] 2014-01-05T06:00:00,
 <xarray.DataArray 'pres_level' ()>
 array(60)
 Coordinates:
     lon      float32 27.0
     lat      float32 -28.5
     time     datetime64[ns] 2014-01-05T06:00:00)

In [3]:
print("Fluxes at present day")
print("Mean at toa", (ref.isel(expt=1,level=toa) * ref.profile_weight).sum())
print("Err  at toa", (new.isel(expt=1,level=toa) * new.profile_weight).sum() - (ref.isel(expt=1,level=toa) * ref.profile_weight).sum())
print("Mean at sfc", (ref.isel(expt=1,level=sfc) * ref.profile_weight).sum())
print("Err  at toa", (new.isel(expt=1,level=sfc) * new.profile_weight).sum() - (ref.isel(expt=1,level=sfc) * ref.profile_weight).sum())

Fluxes at present day
Mean at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.01
    rld             float64 0.0
    profile_weight  float64 0.01053
    rlu             float64 266.0
    rsd             float64 335.1
    rsu             float64 47.37
Err  at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0
    profile_weight  float64 0.0
    rlu             float64 -0.1151
    rsd             float64 0.1126
    rsu             float64 0.2275
Mean at sfc <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 9.862e+04
    rld             float64 313.3
    profile_weight  float64 0.01053
    rlu             float64 397.8
    rsd             float64 246.1
    rsu             float64 30.21
Err  at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.8197
    profile_weight  floa

In [4]:
#
# Forcing - check defintion (which is PI, PD?)
#
ref_f = ref.isel(expt=1) - ref.isel(expt=0)
new_f = new.isel(expt=1) - new.isel(expt=0)

print("Forcing relative to PI")
print("Mean at toa", (ref_f.isel(level=toa) * ref.profile_weight).sum())
print("Err  at toa", (new_f.isel(level=toa) * new.profile_weight).sum() - (ref_f.isel(level=toa) * ref.profile_weight).sum())
print("Mean at sfc", (ref_f.isel(level=sfc) * ref.profile_weight).sum())
print("Err  at toa", (new_f.isel(level=sfc) * new.profile_weight).sum() - (ref_f.isel(level=sfc) * ref.profile_weight).sum())

Forcing relative to PI
Mean at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0
    profile_weight  float64 0.0
    rlu             float64 2.845
    rsd             float64 0.01483
    rsu             float64 0.05815
Err  at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0
    profile_weight  float64 0.0
    rlu             float64 -0.148
    rsd             float64 -0.01483
    rsu             float64 -0.007025
Mean at sfc <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 -2.083
    profile_weight  float64 0.0
    rlu             float64 -0.04165
    rsd             float64 0.5344
    rsu             float64 0.05609
Err  at toa <xarray.Dataset>
Dimensions:         ()
Data variables:
    plev            float64 0.0
    rld             float64 0.0952
    profile_weight  flo

In [5]:
print("Range in errrors in longwave fluxes by experiment")
for v in ["rld", "rlu"]: 
  print(v,     (new.variables[v] - ref.variables[v]).min(dim="site").min(dim="level").values) 
  print("   ", (new.variables[v] - ref.variables[v]).max(dim="site").max(dim="level").values)

print("Range in errrors in shortwave fluxes by experiment")
for v in ["rsd", "rsu"]: 
  print(v, 
        (new.isel(expt=[0,1]).variables[v] - ref.isel(expt=[0,1]).variables[v]).min(dim="site").min(dim="level").values) 
  print("   ", 
        (new.isel(expt=[0,1]).variables[v] - ref.isel(expt=[0,1]).variables[v]).max(dim="site").max(dim="level").values)



Range in errrors in longwave fluxes by experiment
rld [-1.4260254 -1.2331848 -1.3813782 -1.4242249 -1.6044617 -1.4402466
 -1.430481  -1.8424072 -1.3729858 -1.325592  -1.4216309 -1.4463196
 -1.3533325 -1.4832458 -1.4559631 -4.082779  -1.4179077 -1.2713013]
    [3.1961365 3.2493286 3.1679382 3.1982117 3.285736  3.1412659 3.1270447
 3.5472412 3.2499084 3.2149963 3.1922302 3.1704712 3.20578   3.3215332
 3.7028198 2.670807  3.3449097 3.255127 ]
rlu [ -1.7913513  -1.7401428  -1.9126587  -1.7939758  -1.5116577  -1.8092346
  -1.8487244  -2.298462   -1.7626038  -1.7842712  -1.7858887  -1.7650757
  -1.8248596 -18.961182  -18.556183  -18.850464  -65.55286    -1.6076355]
    [1.001831   0.81048584 0.9102783  1.0022583  1.1364441  0.978302
 0.9441223  0.79574585 1.0302124  0.96255493 0.96694946 0.9734802
 0.8695984  0.06921387 0.081604   0.05725098 0.06939697 0.7457886 ]
Range in errrors in shortwave fluxes by experiment
rsd [-2.2943726 -2.36969  ]
    [1.7263031 1.7143707]
rsu [-7.9863424 -7.95059

In [6]:
ref.close()
new.close()